In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [28]:
path_to_data = "/content/drive/My Drive/NLP/trial/"
files_cross = "trial.en-ar.data  trial.en-es.gold  trial.en-ru.data  trial.en-zh.gold  trial.en-ar.gold  trial.en-fr.data  trial.en-ru.gold  trial.en-es.data  trial.en-fr.gold  trial.en-zh.data"
files_cross = files_cross.split('  ')
files_multi = "trial.ar-ar.data  trial.en-en.gold  trial.ru-ru.data  trial.zh-zh.gold  trial.ar-ar.gold  trial.fr-fr.data  trial.ru-ru.gold  trial.en-en.data  trial.fr-fr.gold  trial.zh-zh.data"
files_multi = files_multi.split('  ')
path_train = "/content/drive/My Drive/NLP/"

def calculate_word_index(str,start):
  count = 0
  start = int(start)
  for i in range(start):
    if(str[i] == ' '):
      count = count + 1
  return count

import pandas as pd
train_data_path = path_train + "WiC_dataset/train/"
labels_list = ['F','T']
train_data_xl = path_train + "xlwic_datasets/"
train_data_xn_en = open(train_data_xl + "xlwic_wn_xlingual/train_en.txt",'r').read().split('\n')[:-1]
valid_data_xn_en = open(train_data_xl + "xlwic_wn_xlingual/valid_en.txt",'r').read().split('\n')[:-1]
data_en = open(train_data_path + "train.data.txt",'r').read().split('\n')[:-1]
labels = open(train_data_path+"train.gold.txt",'r').read().split('\n')[:-1]
chinese_data_test = open(train_data_xl + "xlwic_wn_xlingual/chinese_zh/test.txt",'r').read().split('\n')[:-1]
chinese_data = open(train_data_xl + "xlwic_wn_xlingual/chinese_zh/valid.txt",'r').read().split('\n')[:-1]
# print(chinese_data[-1])
french_data = open(train_data_xl + "xlwic_wikt_xlingual/french_fr/test.txt",'r').read().split('\n')[:-1]
xl_wic_data = [valid_data_xn_en,chinese_data,chinese_data_test]
temp = []
for i in data_en:
  splitted = i.split('\t')
  line = []

  for j in range(len(splitted)):
    if(j == 2):
      line.append(int(splitted[j].split('-')[0]))
      line.append(int(splitted[j].split('-')[1]))
    else:
      line.append(splitted[j])
  temp.append(line)

data = temp
for i in xl_wic_data:
  for j in i:
    line = []
    j = j.split('\t')
    # print(j)
    line.append(j[0])
    line.append(j[1])
    line.append(calculate_word_index(j[7],j[4]))
    line.append(calculate_word_index(j[6],j[2]))
    line.append(j[7])
    line.append(j[6])
    
    labels.append(labels_list[int(j[8])])
    data.append(line)
data = pd.DataFrame(data)
data.head()
data.columns = ["word","pos","position1","position2","sent1","sent2"]
data.head()

,word,pos,position1,position2,sent1,sent2
0,carry,V,2,1,You must carry your camping gear .,Sound carries well over water .
1,go,V,2,6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?
2,break,V,0,2,Break an alibi .,The wholesaler broke the container loads into ...
3,cup,N,8,4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .
4,academy,N,1,2,The Academy of Music .,The French Academy .


In [29]:
sentences = data['sentence1'] + ' [SEP] ' + data['sentence2']
labels = pd.DataFrame(labels).replace('F',0)
labels = labels.replace('T',1)
results = labels.to_numpy()

KeyError: ignored

In [5]:
!nvidia-smi

Wed Oct 14 18:30:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [24]:
!pip install transformers

from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [23]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  You must carry your camping gear . [SEP] Sound carries well over water .
Tokenized:  ['you', 'must', 'carry', 'your', 'camping', 'gear', '.', '[SEP]', 'sound', 'carries', 'well', 'over', 'water', '.']
Token IDs:  [10855, 14650, 26913, 12787, 66626, 30238, 119, 102, 14127, 60035, 11327, 10323, 11917, 119]


In [30]:
from sklearn.model_selection import train_test_split
train_data_df,dev_data_df,train_labels,dev_labels = train_test_split(data,labels,test_size = 0.33)

In [44]:
train_labels = pd.DataFrame(train_labels)
train_labels.head()
dev_labels = pd.DataFrame(dev_labels)

In [32]:
train_sentences_1 = list(train_data_df['sent1'])
train_sentences_2 = list(train_data_df['sent2'])

dev_sentences_1 =  list(dev_data_df['sent1'])
dev_sentences_2 = list(dev_data_df['sent2'])


In [33]:
encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))
print(train_sentences_1[3])
print(train_sentences_2[3])
print((encoded_inputs_train['offset_mapping'][3]))
print((encoded_inputs_train['input_ids'][3]))

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

162
<class 'torch.Tensor'>
劉兄手下有不少弟兄、與薊州三霸頗有交情。
看來其他三校要打破政大的「連四霸」紀錄，還真的得加把勁才行！
tensor([[ 0,  0],
        [ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [13, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [17, 18],
        [18, 19],
        [19, 20],
        [ 0,  0],
        [ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [13, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [17, 18],
        [18, 19],
        [19, 20],
        [20, 21],
        [21, 22],
        [22, 23],
        [23, 24],
        [24, 25],
        [25, 26],
        [26, 27],
        [27, 28],
        [28, 29],
   

In [46]:
import numpy as np
train_labels = train_labels.replace({'F' : 0, 'T' : 1})
dev_labels = dev_labels.replace({'F' : 0, 'T' : 1})
train_labels =  torch.from_numpy(train_labels.to_numpy())
dev_labels =  torch.from_numpy(dev_labels.to_numpy())

In [47]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_labels)

In [49]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [53]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-multilingual-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, input_ids, token_type_ids, attention_mask, labels):
        loss, logits = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels=labels)[:2]

        return loss, logits

In [54]:
model = BERTi().to(device)
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

The BERT model has 201 different named parameters.

==== Embedding Layer ====

encoder.bert.embeddings.word_embeddings.weight          (105879, 768)
encoder.bert.embeddings.position_embeddings.weight        (512, 768)
encoder.bert.embeddings.token_type_embeddings.weight        (2, 768)
encoder.bert.embeddings.LayerNorm.weight                      (768,)
encoder.bert.embeddings.LayerNorm.bias                        (768,)

==== First Transformer ====

encoder.bert.encoder.layer.0.attention.self.query.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.self.query.bias        (768,)
encoder.bert.encoder.layer.0.attention.self.key.weight    (768, 768)
encoder.bert.encoder.layer.0.attention.self.key.bias          (768,)
encoder.bert.encoder.layer.0.attention.self.value.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.self.value.bias        (768,)
encoder.bert.encoder.layer.0.attention.output.dense.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.output.dense.bias 

In [55]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

# scheduler??

In [56]:


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss()(output, targets)



In [57]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [58]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [59]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 4
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 5 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy(all_dev_logits, all_dev_labels)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch     5  of    614.    Elapsed: 0:00:02.
  Batch    10  of    614.    Elapsed: 0:00:03.
  Batch    15  of    614.    Elapsed: 0:00:05.
  Batch    20  of    614.    Elapsed: 0:00:06.
  Batch    25  of    614.    Elapsed: 0:00:08.
  Batch    30  of    614.    Elapsed: 0:00:09.
  Batch    35  of    614.    Elapsed: 0:00:11.
  Batch    40  of    614.    Elapsed: 0:00:12.
  Batch    45  of    614.    Elapsed: 0:00:14.
  Batch    50  of    614.    Elapsed: 0:00:15.
  Batch    55  of    614.    Elapsed: 0:00:17.
  Batch    60  of    614.    Elapsed: 0:00:18.
  Batch    65  of    614.    Elapsed: 0:00:20.
  Batch    70  of    614.    Elapsed: 0:00:21.
  Batch    75  of    614.    Elapsed: 0:00:23.
  Batch    80  of    614.    Elapsed: 0:00:24.
  Batch    85  of    614.    Elapsed: 0:00:26.
  Batch    90  of    614.    Elapsed: 0:00:27.
  Batch    95  of    614.    Elapsed: 0:00:29.
  Batch   100  of    614.    Elapsed: 0:00:30.
  Batch   105  of